# Notebook 13: Finding Good Reference Units

When working with physical problems, the numbers can get unwieldy. Earth's mass is 6×10²⁴ kg, mantle convection happens over millions of years, and we need to handle both large and tiny numbers simultaneously.

Good reference units make your problem easier to work with by choosing natural scales for your system.

In [1]:
import nest_asyncio

nest_asyncio.apply()

import numpy as np
import underworld3 as uw

## The Problem: Mixed Scales

Consider a mantle convection problem with typical parameters:

In [2]:
# Physical parameters (SI units)
mantle_depth = 2900 * uw.units.km
mantle_viscosity = 1e19 * uw.units("Pa.s")
plate_velocity = 5 * uw.units.cm / uw.units.year
mantle_temperature = 1500 * uw.units.K
density = 3000 * uw.units("kg/m^3")
diffusivity = 1.0e-6 * uw.units("m^2/s")

Those SI units mix huge numbers (viscosity) with tiny ones (velocity). This causes:
- Hard to read and verify
- Numerical rounding errors
- Mental overhead converting back and forth

## Solution: Natural Reference Scales

Choose units that make your numbers order-one. For mantle convection:

In [3]:
# Set up the model with reference quantities
model = uw.Model()
model.set_reference_quantities(
    length_scale=mantle_depth,
    temperature_scale=mantle_temperature,
    time=1*uw.units("10Gyr"),
    viscosity=1*uw.units("exaPa.s"),
)

# Check what units the model chose
model.get_model_base_units()

{'length': '1.0 megameter',
 'time': '100.00000000000001 petasecond',
 'mass': '0.9999999999999999 zettagram',
 'temperature': '1.0 kilokelvin'}

In [4]:
print(model.get_scale_summary())

Fundamental Scales Summary:

Length Scale: 1.0 megameter
    → SI: 1000000 meter
    → Model: 1.0 model unit = 1.0 megameter (≡ 1000000 meter)
  - From: derived from dimensional analysis
  - Derived From Dimensional Analysis

Time Scale: 100.00000000000001 petasecond
    → SI: 100000000000000000 second
    → Model: 1.0 model unit = 100.00000000000001 petasecond (≡ 100000000000000000 second)
  - From: derived from dimensional analysis
  - Derived From Dimensional Analysis

Mass Scale: 0.9999999999999999 zettagram
    → Friendly: 1000000.0 petagram
    → SI: 1000000000000000000 kilogram
    → Model: 1.0 model unit = 0.9999999999999999 zettagram (≡ 1000000000000000000 kilogram)
  - From: viscosity
  - Makes: viscosity ≈ 1.0 in model units

Temperature Scale: 1.0 kilokelvin
    → SI: 1000 kelvin
    → Model: 1.0 model unit = 1.0 kilokelvin (≡ 1000 kelvin)
  - From: temperature_scale
  - Makes: temperature_scale ≈ 1.0 in model units


In [5]:
model.to_model_units(mantle_viscosity)

UWQuantity(1e+24, 'zettagram / megameter / petasecond')

In [6]:
model.to_model_units(plate_velocity)

UWQuantity(158.44043907014472, 'megameter / petasecond')

In [7]:
model.to_model_units(density) # .to_compact()

UWQuantity(3000.0, 'zettagram / megameter ** 3')

In [8]:
model.to_model_units(mantle_temperature)

UWQuantity(1.5, 'kilokelvin')

In [9]:
model.to_model_units(10 * uw.units("m/s/s"))

UWQuantity(1e+29, 'megameter / petasecond ** 2')

Now when we work in these units, our numbers are much friendlier:

In [10]:
# Convert to model units
depth_model = model.to_model_units(mantle_depth)
velocity_model = model.to_model_units(plate_velocity)

print(f"Depth in model units:    {depth_model}")
print(f"Velocity in model units: {velocity_model}")

Depth in model units:    2.9 megameter
Velocity in model units: 158.44043907014472 megameter / petasecond


In [11]:
print(model.to_model_units(plate_velocity))
print(plate_velocity)
print(plate_velocity.to("nm/s"))
print(plate_velocity.to("Mm/ns"))

158.44043907014472 megameter / petasecond
5.0 centimeter / year
1.5844043907014471 nanometer / second
1.584404390701447e-24 megameter / nanosecond


In [12]:
model.to_model_units(1 * uw.units("Gyear"))

UWQuantity(0.315576, 'petasecond')

In [13]:
(1e9 * uw.units("Zs")).to("s")

<Quantity(1e+30, 'second')>

In [14]:
model.to_model_units(mantle_viscosity)

UWQuantity(1e+24, 'zettagram / megameter / petasecond')

## Easy Unit Conversions

Once you've set reference quantities, converting between systems is straightforward:

In [15]:
# Work with different units easily
plate_velocity_SI = plate_velocity.to("m/s")
plate_velocity_mmy = plate_velocity.to("mm/year")

print(f"Plate velocity:")
print(f"  Original:     {plate_velocity}")
print(f"  SI units:     {plate_velocity_SI}")
print(f"  mm/year:      {plate_velocity_mmy}")
print(f"  Model units:  {velocity_model}")

Plate velocity:
  Original:     5.0 centimeter / year
  SI units:     1.5844043907014474e-09 meter / second
  mm/year:      50.0 millimeter / year
  Model units:  158.44043907014472 megameter / petasecond


## Choosing Good Reference Quantities

Pick values that are typical for your problem:

**Length**: 
- Mantle convection → depth of mantle (2900 km)
- Plate tectonics → plate thickness (100 km)
- Laboratory → sample size (10 cm)

**Temperature**:
- Temperature difference across the domain
- Or a characteristic temperature (like mantle temperature)

**Viscosity/Density**:
- Representative material properties

The model uses these to derive derived scales (like time = length²/diffusivity).

## Working with a Mesh

Meshes automatically use the model's coordinate units:

In [16]:
# Create a mesh - it uses model coordinate units
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0),
    maxCoords=(1000.0, mantle_depth.magnitude),  # 1000 km × 2900 km
    cellSize=200.0,
    qdegree=2,
)

print(f"Mesh coordinate units: {mesh.units}")
print(f"Domain size: {mesh.X.coords[:, 0].max()} × {mesh.X.coords[:, 1].max()}")

Mesh coordinate units: kilometer
Domain size: 1000000000.0 kilometer × 2900000000.0 kilometer


In [17]:
# Create variables with physical units
temperature = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")
velocity = uw.discretisation.MeshVariable("v", mesh, 2, degree=2, units="cm/year")

# Units are preserved
temperature.units

<Unit('kelvin')>

## Summary

**Why use reference units?**
- Makes numbers easier to read and verify
- Reduces numerical errors
- Natural for your problem domain

**How to choose them:**
- Pick characteristic values for your system
- Length: domain size or characteristic length scale
- Temperature: temperature difference or characteristic temperature
- Material properties: representative values

**Working with the model:**
```python
model.set_reference_quantities(
    domain_depth=2900 * uw.units.km,
    mantle_temperature=1500 * uw.units.K,
    mantle_viscosity=1e21 * uw.units("Pa.s"),
)

# Convert easily
model.to_model_units(my_quantity)
```

The unit system handles all conversions automatically - you just work with physical quantities and the right thing happens.

## Try It Yourself

Experiment with different reference quantities:

```python
# Laboratory scale problem
lab_model = uw.Model()
lab_model.set_reference_quantities(
    domain_depth=10 * uw.units.cm,
    reference_temperature=100 * uw.units.K,
    reference_viscosity=1 * uw.units("Pa.s"),
)

# How do mantle values look in lab units?
lab_model.to_model_units(mantle_depth)
lab_model.to_model_units(mantle_viscosity)
```